In [ ]:
def gaussian_kernel(size=21, sigma=2.5):
    """Create a 2D Gaussian PSF kernel, normalized to sum=1."""
    assert size % 2 == 1, "Kernel size should be odd."
    ax = np.arange(-(size // 2), size // 2 + 1)
    xx, yy = np.meshgrid(ax, ax)
    k = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    k /= np.sum(k) #energy =1, the energy of PSF should equal to the energy of delta function
    return k.astype(np.float32)


### Explanation `gaussian_kernel(size=21, sigma=2.5)`

This function constructs a **2D Gaussian PSF (blur kernel)** and then **normalizes it so the discrete sum equals 1**.  
A standard forward imaging model is:

$
y = k * x + n
$

where:
- $x$ is the sharp image,
- $k$ is the PSF / blur kernel,
- $n$ is noise,
- $*$ denotes 2D convolution.

---


### 1) Function header
- `size` sets the kernel shape as `size × size`.
- `sigma` controls blur width (larger $\sigma$ → more blur).

### 2) Ensure odd size
```python
assert size % 2 == 1, "Kernel size should be odd."
````

This forces `size` to be odd so the kernel has a well-defined center pixel.

### 3) Build coordinate grid

```python
ax = np.arange(-(size // 2), size // 2 + 1)
xx, yy = np.meshgrid(ax, ax)
```

This creates coordinates:
$$
a = {-\lfloor \tfrac{size}{2}\rfloor, \ldots, 0, \ldots, \lfloor \tfrac{size}{2}\rfloor}
$$

and then 2D grids $$x_{ij}, y_{ij}$$ (stored in `xx, yy`).

### 4) Compute the (unnormalized) 2D Gaussian

```python
k = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
```

This implements:
$$
k[i,j] = \exp\left(-\frac{x_{ij}^2 + y_{ij}^2}{2\sigma^2}\right)
$$

Note: The continuous Gaussian often includes a prefactor $$\frac{1}{2\pi\sigma^2}$$, but we omit it here because we will normalize discretely.

### 5) Normalize to unit-sum

```python
k /= np.sum(k)
```

After this step:
$$
\sum_{i,j} k[i,j] = 1
$$

#### Why normalize by the sum?

Because this ensures **brightness preservation** under convolution.
If the input is a constant image $$x[i,j] = c$$, then:

$$
(k * x)[i,j] = \sum_{u,v} k[u,v] , x[i-u, j-v]
= \sum_{u,v} k[u,v] , c
= c \sum_{u,v} k[u,v]
= c
$$

So the blur does **not** change the overall intensity level.

#### Important clarification about the comment “energy = 1”

* The condition $$\sum k = 1$$ is **unit-sum / unit DC gain**, not “energy”.
* In many signal-processing contexts, “energy” refers to the L2 energy:

$$
|k|_2^2 = \sum_{i,j} k[i,j]^2
$$

For PSFs in imaging, **unit-sum** is typically what we want because it corresponds to conserving total intensity (the PSF behaves like a discrete probability distribution).

### 6) Return as float32

```python
return k.astype(np.float32)
```

This returns the kernel in `float32` for speed and memory efficiency.

---

## Summary

The function creates:
$$
k[i,j] \propto \exp\left(-\frac{i^2 + j^2}{2\sigma^2}\right)
$$

then normalizes it so that:
$$
\sum_{i,j} k[i,j] = 1
$$

which makes convolution preserve constant brightness.

---

## Optional: If you truly wanted "unit energy" instead of "unit sum"

If you wanted $$|k|_2 = 1$$, you would do:

$$
k \leftarrow \frac{k}{\sqrt{\sum_{i,j} k[i,j]^2}}
$$

But for blur PSFs, **unit-sum normalization** is the standard choice.

```
```


In [ ]:
def psf2otf(psf, shape):
    """
    Convert a spatial PSF to OTF (FFT of padded+shifted PSF), so that:
    circular_conv(x, psf) == ifft2( fft2(x) * OTF )
    """
    #in order to apply frequency filtering, the size of PSF spectrum and the size of image spectrm should be the same
    psf_pad = np.zeros(shape, dtype=np.float32)
    kh, kw = psf.shape
    psf_pad[:kh, :kw] = psf

    # Shift PSF center to (0,0) for correct circular convolution in FFT domain
    psf_pad = np.roll(psf_pad, -kh // 2, axis=0)
    psf_pad = np.roll(psf_pad, -kw // 2, axis=1)

    return np.fft.fft2(psf_pad)

### Explanation  `psf2otf(psf, shape)`

This function converts a **spatial-domain PSF** (a small blur kernel) into its **frequency-domain representation**, called the **OTF** (Optical Transfer Function).  
The key identity it enables is:

$$
x \circledast k \;=\; \mathcal{F}^{-1}\Big( \mathcal{F}(x)\,\odot\,\mathcal{F}(k)\Big)
$$

where:
- $\circledast$ = **circular convolution** (periodic boundary condition),
- $\mathcal{F}(\cdot)$ / $\mathcal{F}^{-1}(\cdot)$ = 2D FFT / inverse FFT,
- $\odot$ = elementwise multiplication.

So once we compute $K=\mathrm{OTF}=\mathcal{F}(k)$, we can do fast convolution by multiplying in the frequency domain.

---

## Why we need padding to `shape`

The FFT-based multiplication requires **same-size arrays**:

$$
X = \mathcal{F}(x)\in\mathbb{C}^{H\times W},\quad
K = \mathcal{F}(k)\in\mathbb{C}^{H\times W}
$$

If the PSF $k$ is smaller (e.g. $21\times 21$) but the image is $H\times W$, we must embed/pad the PSF into an $H\times W$ array before FFT, otherwise $X\odot K$ is undefined.

---

## Line-by-line explanation

### 1) Allocate a zero-padded array of the target shape
```python
psf_pad = np.zeros(shape, dtype=np.float32)
kh, kw = psf.shape
psf_pad[:kh, :kw] = psf
````

This creates $k_{\text{pad}}\in\mathbb{R}^{H\times W}$ such that the PSF values are copied into the **top-left corner**:

* Before shifting, the PSF center is **not** at the origin index $(0,0)$ (it is typically near $(\lfloor kh/2\rfloor,\lfloor kw/2\rfloor)$).

### 2) Shift the PSF center to frequency-domain “origin” (0,0)

```python
psf_pad = np.roll(psf_pad, -kh // 2, axis=0)
psf_pad = np.roll(psf_pad, -kw // 2, axis=1)
```

This step is crucial for making the FFT correspond to **circular convolution with a centered kernel**.

#### What `np.roll` does

`np.roll(A, s, axis)` circularly shifts array `A` by `s` along the given axis:

* pixels that “fall off” one end wrap around to the other end (periodic shift).

#### Why shift by `-kh//2` and `-kw//2`?

In spatial convolution, we usually think the PSF is centered at its middle index.
But the FFT assumes the impulse response is aligned so that the “zero-lag” sample is at index $(0,0)$.

So we perform:

$$
k_{\text{shift}}[i,j] = k_{\text{pad}}\Big((i+\lfloor kh/2\rfloor)\bmod H,; (j+\lfloor kw/2\rfloor)\bmod W\Big)
$$

This is conceptually the same as applying `ifftshift` (depending on conventions). After this shift, the FFT produces an OTF that correctly matches the intended circular convolution.

**If you do NOT shift**, the convolution result will look spatially “mis-registered” (often appearing split/shifted), because the kernel’s center is effectively interpreted as being at the corner.

<div style="color:red; font-weight:bold;">
My note: This is closely related to FFT princple. FFT assumes periodic function, which extends the original function. This is the same case for both PSF and original function. The "origin point" is set at the upleft corner. But remeber the real function is actually a "infinite large" image with multiple copys of the original image.If you donot shift the center of psf to the original point. The final result will also be a shifted version of corrected image. Since we can only observe the "infinite" image in a finite size window. Thus we can get a misaligned image.But remeber, that is a shifted version of infinite image.  
</div>


### 3) Take the 2D FFT to get the OTF

```python
return np.fft.fft2(psf_pad)
```

This returns:

$$
K(u,v) = \mathcal{F}{k_{\text{shift}}}(u,v)
$$

which is the **OTF** used in frequency-domain filtering:
$$
Y = X \odot K
$$

---

## Summary

* **Pad** PSF to image size so FFT-domain multiplication is possible.
* **Roll/shift** so the PSF center is at index $$(0,0)$$, matching FFT’s circular-convolution convention.
* **FFT2** produces the OTF, enabling fast circular convolution by:
  $$
  x \circledast k = \mathcal{F}^{-1}\big(\mathcal{F}(x)\odot \mathcal{F}(k)\big)
  $$

```
```


In [ ]:
def circular_conv2d(x, psf):
    """2D circular convolution using FFT."""
    # typical circular convolution,when you use two same size frequency domains times each other
    # this is called cicular convolution.
    K = psf2otf(psf, x.shape)
    return np.real(np.fft.ifft2(np.fft.fft2(x) * K)).astype(np.float32)
def forward_model(x, psf, sigma_noise, seed=0):
    """
    y = k*x + n, n ~ N(0, sigma^2)
    x is assumed in [0,1].
    """
    rng = np.random.default_rng(seed)
    y_blur = circular_conv2d(x, psf)
    n = rng.normal(0.0, sigma_noise, size=x.shape).astype(np.float32)
    y = y_blur + n
    return y, y_blur


### Explanation `circular_conv2d` and `forward_model`

These two functions implement a common **imaging forward model**:

$$
y = k \circledast x + n,\quad n \sim \mathcal{N}(0,\sigma^2)
$$

where:
- $x \in [0,1]$ is the clean image,
- $k$ (your `psf`) is the blur kernel / PSF,
- $\circledast$ denotes **2D circular convolution** (periodic boundary condition),
- $n$ is additive white Gaussian noise (AWGN),
- $y$ is the observed (blurred + noisy) measurement.

---

## 1) `circular_conv2d(x, psf)`

```python
def circular_conv2d(x, psf):
    """2D circular convolution using FFT."""
    K = psf2otf(psf, x.shape)
    return np.real(np.fft.ifft2(np.fft.fft2(x) * K)).astype(np.float32)
````

### What it computes

This function computes:

$$
y = x \circledast k
$$

Using the convolution theorem (for **circular** convolution):

$$
\mathcal{F}(x \circledast k) ;=; \mathcal{F}(x)\odot \mathcal{F}(k)
$$

So the algorithm is:

1. Compute the OTF (FFT of the padded+shifted PSF):
   $$
   K = \mathcal{F}(k_{\text{shift}})
   $$
2. Compute FFT of the image:
   $$
   X = \mathcal{F}(x)
   $$
3. Multiply in frequency domain:
   $$
   Y = X \odot K
   $$
4. Inverse FFT to get back to spatial domain:
   $$
   y = \mathcal{F}^{-1}(Y)
   $$

### Why is this called “circular convolution”?

Because FFT-based multiplication corresponds to **convolution on a periodic domain**:

* The image is assumed to wrap around at the borders.
* Mathematically, indices are interpreted modulo the image size $$H,W$$.

A precise discrete definition is:

$$
(x \circledast k)[i,j] = \sum_{u=0}^{H-1}\sum_{v=0}^{W-1} x[u,v]; k[(i-u)\bmod H,; (j-v)\bmod W]
$$

This is different from **linear convolution**, which would require padding the image (e.g., zero-padding) to avoid wrap-around artifacts.

### Why `np.real(...)`?

Due to floating point round-off, `ifft2` may return tiny imaginary parts (e.g., $$10^{-12}i$$) even when the true result is real.
So we take:

$$
\Re{y}
$$

### Why `.astype(np.float32)`?

For memory/speed consistency and to match typical imaging pipelines.

---

## 2) `forward_model(x, psf, sigma_noise, seed=0)`

```python
def forward_model(x, psf, sigma_noise, seed=0):
    """
    y = k*x + n, n ~ N(0, sigma^2)
    x is assumed in [0,1].
    """
    rng = np.random.default_rng(seed)
    y_blur = circular_conv2d(x, psf)
    n = rng.normal(0.0, sigma_noise, size=x.shape).astype(np.float32)
    y = y_blur + n
    return y, y_blur
```

### What it computes

It implements:

1. **Blur (circular convolution)**:
   $$
   y_{\text{blur}} = k \circledast x
   $$
2. **Additive Gaussian noise** (i.i.d. per pixel):
   $$
   n[i,j] \sim \mathcal{N}(0,\sigma^2)
   $$
3. **Observed measurement**:
   $$
   y = y_{\text{blur}} + n
   $$

### The role of `seed`

```python
rng = np.random.default_rng(seed)
```

This makes the noise **reproducible**: with the same `seed`, you get the same random noise realization.

### Why return both `(y, y_blur)`?

* `y` is what you would “measure” in a realistic imaging scenario.
* `y_blur` is useful for debugging/analysis (separating blur-only effects from noise effects).

---

## Summary

* `circular_conv2d`: computes circular convolution efficiently via FFT:

$$
x \circledast k = \mathcal{F}^{-1}\big(\mathcal{F}(x)\odot \mathcal{F}(k)\big)
$$

* `forward_model`: adds Gaussian noise to simulate measurement:

$$
y = k \circledast x + n,\quad n\sim\mathcal{N}(0,\sigma^2)
$$

```
```


In [ ]:
def tikhonov_deconv(y, psf, lam):
    """
    argmin_x 0.5||k*x - y||^2 + 0.5*lam||x||^2
    Closed-form in Fourier domain:
    X = conj(K)*Y / (|K|^2 + lam)
    """
    K = psf2otf(psf, y.shape)
    Y = np.fft.fft2(y)
    X = np.conj(K) * Y / (np.abs(K) ** 2 + lam)
    x_hat = np.real(np.fft.ifft2(X)).astype(np.float32)
    return np.clip(x_hat, 0.0, 1.0)

## Explanation `tikhonov_deconv(y, psf, lam)`

This function performs **Tikhonov-regularized deconvolution** (also called **ridge-regularized least squares**) under a **circular convolution** model.

You assume the observation model is approximately:
- blur: $y \approx k \circledast x$
- (optionally with noise)

and you reconstruct $x$ by solving:

$$
\hat{x}=\arg\min_x \frac{1}{2}\|k\circledast x - y\|_2^2 \;+\; \frac{\lambda}{2}\|x\|_2^2.
$$

- The first term enforces data fidelity (match the blurred image).
- The second term penalizes large energy in $x$ and stabilizes inversion.
- $\lambda>0$ controls the trade-off: larger $\lambda$ → smoother / less noisy but more biased.

---

## Key idea: diagonalization by FFT (circular convolution)

For circular convolution, the convolution operator becomes multiplication in the Fourier domain:

- Let $K=\mathcal{F}(k)$ be the OTF, $X=\mathcal{F}(x)$, $Y=\mathcal{F}(y)$.
- Then $k\circledast x \leftrightarrow K\odot X$.

So the optimization decouples frequency-by-frequency:

$$
\hat{X}(u,v) = \arg\min_{X(u,v)} \frac{1}{2}|K(u,v)X(u,v)-Y(u,v)|^2 + \frac{\lambda}{2}|X(u,v)|^2.
$$

This is a scalar complex ridge regression. Setting derivative to zero gives the closed form:

$$
\hat{X}(u,v)=\frac{K(u,v)^{*}\,Y(u,v)}{|K(u,v)|^2+\lambda}.
$$

That is exactly the formula in the docstring.



---

### 1) Derivation in matrix form (ridge regression / Tikhonov)

Write the circular convolution as a **linear operator**:

- Let $x,y\in\mathbb{R}^N$ be vectorized images.
- Let $A\in\mathbb{R}^{N\times N}$ be the (block-)circulant matrix representing circular convolution by $k$.
  Then $Ax = k\circledast x$.

The objective is:

$$
J(x)=\frac{1}{2}\|Ax-y\|_2^2+\frac{\lambda}{2}\|x\|_2^2.
$$

Compute the gradient (using standard least-squares derivatives):

- For $\frac{1}{2}\|Ax-y\|_2^2$, the gradient is $A^\top(Ax-y)$.
- For $\frac{\lambda}{2}\|x\|_2^2$, the gradient is $\lambda x$.

So:

$$
\nabla J(x)=A^\top(Ax-y)+\lambda x.
$$

At the minimizer $\hat{x}$, set $\nabla J(\hat{x})=0$:

$$
A^\top(A\hat{x}-y)+\lambda\hat{x}=0
\quad\Rightarrow\quad
(A^\top A+\lambda I)\hat{x}=A^\top y.
$$

Assuming $\lambda>0$, the matrix $(A^\top A+\lambda I)$ is positive definite, so the solution is unique:

$$
\hat{x}=(A^\top A+\lambda I)^{-1}A^\top y.
$$

This is exactly **ridge regression** (same algebra, different application domain).

---

### 2) Why the Fourier-domain formula appears (diagonalization of circular convolution)

For **circular** convolution, $A$ is (block-)circulant, and circulant matrices are diagonalized by the DFT:

$$
A = F^{-1}\,\mathrm{diag}(K)\,F,
$$

where:
- $F$ is the (2D) DFT operator (implemented by `fft2`),
- $\mathrm{diag}(K)$ is a diagonal operator whose diagonal entries are the OTF values $K(u,v)$,
- $K=\mathcal{F}(k)$.

Also, the adjoint corresponds to conjugation in the Fourier domain:

$$
A^\top \;\text{(or }A^H\text{ in complex form)}\;=\;F^{-1}\,\mathrm{diag}(K^*)\,F.
$$

Now plug these into the normal equation:

$$
(A^H A+\lambda I)\hat{x}=A^H y.
$$

Left side:

$$
A^H A
= \left(F^{-1}\mathrm{diag}(K^*)F\right)\left(F^{-1}\mathrm{diag}(K)F\right)
= F^{-1}\mathrm{diag}(|K|^2)F.
$$

So:

$$
(A^H A+\lambda I)
= F^{-1}\mathrm{diag}(|K|^2+\lambda)\,F.
$$

Right side:

$$
A^H y = F^{-1}\mathrm{diag}(K^*)\,F y.
$$

Let $Y=Fy$ and $\hat{X}=F\hat{x}$. Multiply both sides by $F$ (using $FF^{-1}=I$):

$$
\mathrm{diag}(|K|^2+\lambda)\,\hat{X}=\mathrm{diag}(K^*)\,Y.
$$

This decouples into independent scalar equations for each frequency bin $(u,v)$:

$$
(|K(u,v)|^2+\lambda)\,\hat{X}(u,v)=K(u,v)^*\,Y(u,v).
$$

Thus:

$$
\hat{X}(u,v)=\frac{K(u,v)^*\,Y(u,v)}{|K(u,v)|^2+\lambda}.
$$

That is the “ridge/Tikhonov” closed-form used in your code.

---

### 3) Why moving from spatial domain to Fourier domain is *valid* (not a trick)

It is valid because the FFT is (up to normalization) a **unitary change of basis**. Conceptually:

- You are not changing the problem, only rewriting it in a basis where circular convolution becomes diagonal.
- Norms are preserved under a unitary transform (Parseval’s theorem):

$$
\|x\|_2^2 \propto \|\mathcal{F}(x)\|_2^2,\qquad
\|Ax-y\|_2^2 \propto \|\mathcal{F}(Ax-y)\|_2^2.
$$

So minimizing the objective in spatial domain is equivalent to minimizing it in Fourier domain, and the Fourier domain is easier because it becomes “one scalar problem per frequency”.

**Important condition:** this exact diagonalization requires **circular convolution** (periodic boundary).  
If your true physics is **linear convolution with zero boundary**, FFT still can be used, but you must pad appropriately or accept wrap-around artifacts; otherwise the Fourier-domain closed form is solving a *slightly different* boundary-condition model.



---

## Line-by-line explanation

### 1) Convert PSF to OTF with matching size
```python
K = psf2otf(psf, y.shape)
````

`psf2otf` pads and shifts the PSF to size `y.shape`, then FFTs it to get $K=\mathcal{F}(k)$.

### 2) FFT of the observed image

```python
Y = np.fft.fft2(y)
```

This computes $Y=\mathcal{F}(y)$.

### 3) Closed-form Tikhonov solution (frequency-wise)

```python
X = np.conj(K) * Y / (np.abs(K) ** 2 + lam)
```

This implements the elementwise formula:

* `np.conj(K)` is $K^*$ (complex conjugate).
* `np.abs(K)**2` is $|K|^2$.
* Division is elementwise, so each frequency bin is filtered by:

$$
H(u,v)=\frac{K(u,v)^*}{|K(u,v)|^2+\lambda}.
$$

This filter is sometimes called a **Wiener-like** (but not identical to Wiener) inverse filter: it avoids division by very small $|K|$ values by adding $\lambda$.

### 4) Inverse FFT back to spatial domain

```python
x_hat = np.real(np.fft.ifft2(X)).astype(np.float32)
```

Compute $\hat{x}=\Re{\mathcal{F}^{-1}(\hat{X})}$ and drop tiny imaginary numerical errors.

### 5) Clip to a valid intensity range

```python
return np.clip(x_hat, 0.0, 1.0)
```

Because you assume the true image $x\in[0,1]$, clipping keeps the reconstruction in the same range (useful especially when noise/overshoot produces negatives or values > 1).

---

## Intuition for $\lambda$

* If $\lambda \to 0$, the solution approaches the naive inverse filter: $\hat{X}\approx Y/K$ (unstable when $K$ has small magnitudes).
* If $\lambda$ is large, high-frequency components (where $|K|$ is small) are strongly suppressed, reducing noise amplification but also losing detail.

A simple way to see the stabilization is that the denominator $|K|^2+\lambda$ is never too close to zero.

```
```


In [ ]:
def gd_deconv(y, psf, lam, x_true=None, max_iters=200, patience=20):
    """
    Gradient descent on:
      f(x)=0.5||k*x - y||^2 + 0.5*lam||x||^2

    With circular convolution:
      grad = k^T*(k*x - y) + lam*x
    where k^T corresponds to conj(K) in Fourier domain.

    Step size alpha chosen via Lipschitz bound:
      L = max(|K|^2) + lam, alpha = 1/L

    Early stopping (optional):
      if x_true provided, stop when PSNR doesn't improve for 'patience' iters.
    """
    K = psf2otf(psf, y.shape)
    Y = np.fft.fft2(y)

    L = float(np.max(np.abs(K) ** 2).real + lam)
    alpha = 1.0 / L

    x = y.copy().astype(np.float32)  # init
    best_psnr = -np.inf
    best_x = x.copy()
    bad_count = 0

    psnr_trace = []

    for it in range(max_iters):
        X = np.fft.fft2(x)
        resid_f = K * X - Y                        # FFT(k*x - y)
        grad_data = np.real(np.fft.ifft2(np.conj(K) * resid_f))  # k^T*(k*x - y)
        grad = grad_data + lam * x

        x = x - alpha * grad
        x = np.clip(x, 0.0, 1.0)

        if x_true is not None:
            p = peak_signal_noise_ratio(x_true, x, data_range=1.0)
            psnr_trace.append(p)
            if p > best_psnr + 1e-6:
                best_psnr = p
                best_x = x.copy()
                bad_count = 0
            else:
                bad_count += 1
            if bad_count >= patience:
                break

    return best_x if x_true is not None else x, alpha, np.array(psnr_trace, dtype=np.float32)


## Explanation: `gd_deconv(y, psf, lam, ...)`

This function solves a **Tikhonov-regularized deconvolution** problem using **gradient descent** (instead of the closed-form Fourier solution).

It minimizes the objective:

$$
f(x)=\frac{1}{2}\|k\circledast x - y\|_2^2+\frac{\lambda}{2}\|x\|_2^2,
$$

where:
- $y$ is the observed blurred/noisy image,
- $k$ is the PSF (blur kernel),
- $\circledast$ is **circular convolution**,
- $\lambda>0$ is the Tikhonov (ridge) regularization weight.

---

## 1) Gradient formula (why `conj(K)` appears)

Define the linear operator $A$ as circular convolution by $k$: $Ax = k\circledast x$.
Then:

- data term: $\frac{1}{2}\|Ax-y\|_2^2$ has gradient $A^T(Ax-y)$ (or $A^H(Ax-y)$ in complex notation),
- regularizer: $\frac{\lambda}{2}\|x\|_2^2$ has gradient $\lambda x$.

So the full gradient is:

$$
\nabla f(x)=A^T(Ax-y)+\lambda x.
$$

For circular convolution, $A$ is diagonalized by FFT:
- $Ax \leftrightarrow K\odot X$,
- $A^T(\cdot)$ corresponds to multiplication by $K^*$ (complex conjugate) in the Fourier domain.

That is why the code uses `np.conj(K)`.

---

## 2) What each block of code is doing

### (a) Precompute OTF and FFT of the observation
```python
K = psf2otf(psf, y.shape)
Y = np.fft.fft2(y)
````

Here $K=\mathcal{F}(k)$ and $Y=\mathcal{F}(y)$.

### (b) Step size from a Lipschitz bound

```python
L = float(np.max(np.abs(K) ** 2).real + lam)
alpha = 1.0 / L
```

For gradient descent on a function with $L$-Lipschitz gradient, a safe constant step size is $\alpha \le 1/L$.

In this quadratic problem, the Hessian is $A^T A + \lambda I$.
Under circular convolution, the eigenvalues of $A^T A$ are exactly $|K(u,v)|^2$, so the largest eigenvalue is:

$$
\lambda_{\max}(A^T A)=\max_{u,v}|K(u,v)|^2.
$$

Therefore a valid Lipschitz constant is:

$$
L=\max_{u,v}|K(u,v)|^2+\lambda,
$$

and the code chooses $\alpha = 1/L$.

### (c) Initialization and early-stopping bookkeeping

```python
x = y.copy().astype(np.float32)  # init
best_psnr = -np.inf
best_x = x.copy()
bad_count = 0
psnr_trace = []
```

* Initialize $x_0=y$ (reasonable baseline: start from the blurred/noisy image).
* If `x_true` is provided, track PSNR for early stopping.

### (d) The gradient descent loop

```python
X = np.fft.fft2(x)
resid_f = K * X - Y
grad_data = np.real(np.fft.ifft2(np.conj(K) * resid_f))
grad = grad_data + lam * x
```

Interpretation:

* Current estimate in Fourier domain: $X=\mathcal{F}(x)$.
* Residual in Fourier domain:

  * $K\odot X$ corresponds to $\mathcal{F}(k\circledast x)$,
  * so $K\odot X - Y = \mathcal{F}(k\circledast x - y)$.
* Apply adjoint convolution (the $A^T$ step):

  * multiply by $K^*$ then inverse FFT:

The data-term gradient is:

$$
\nabla_x\left(\frac{1}{2}|k\circledast x-y|_2^2\right)=k^T\circledast (k\circledast x-y),
$$

and in frequency domain this becomes:

$$
\mathcal{F}\{\text{grad\_data}\}
= K^{*}\odot\bigl(K\odot X - Y\bigr).
$$

Then add the regularizer gradient $\lambda x$.

### (e) Update and projection (clipping)

```python
x = x - alpha * grad
x = np.clip(x, 0.0, 1.0)
```

This is:

* gradient descent step: $x_{t+1}=x_t-\alpha\nabla f(x_t)$,
* then a simple projection to the valid intensity range $[0,1]$.

Clipping is **not** part of the original unconstrained Tikhonov problem; it’s a practical constraint reflecting that pixel intensities are assumed to lie in $[0,1]$.

### (f) Early stopping with PSNR (optional)

```python
p = peak_signal_noise_ratio(x_true, x, data_range=1.0)
...
if bad_count >= patience: break
```

* PSNR is computed as:
  $$
  \mathrm{PSNR}(x_{\text{true}},x)=10\log_{10}\left(\frac{\mathrm{MAX}^2}{\mathrm{MSE}}\right),
  $$
  where $\mathrm{MAX}=1$ because `data_range=1.0`, and $\mathrm{MSE}=\frac{1}{N}|x-x_{\text{true}}|_2^2$.
* If PSNR does not improve for `patience` iterations, stop and return the best encountered solution.

---

## 3) Outputs

```python
return best_x if x_true is not None else x, alpha, np.array(psnr_trace, dtype=np.float32)
```

* If `x_true` is given, returns:

  * `best_x`: the iterate with highest PSNR,
  * `alpha`: the step size used,
  * `psnr_trace`: PSNR over iterations.
* Otherwise returns:

  * the final `x`,
  * `alpha`,
  * an empty PSNR trace array.

---

```
```

